In [ ]:
import json
timerSum = {"ch":{"su":{"p1":{"on":"10:00","off":"10:30"}}}}
outfile=open("myfile.json","w")
json.dump(timerSum, outfile, indent=6)

t={}
for c in range(4):
  kc="c"+ str(c)
  t[kc]={}
  for d in range(7):
    kd="d"+str(d)
    t[kc][kd]={}
    for p in range(4) :
      kp="p"+str(p)
      t[kc][kd][kp]={"on":"","off":""}

pj = json.dumps(t, indent=4)
print(pj)
print(t)

outfile.close()

In [ ]:
import json
f=open('params.json',)
data = json.load(f)

print(data)
# for i in data['ch']:
#     print(i)

f.close()

In [ ]:
from flask import Flask
from flask.templating import render_template
app = Flask('app')

@app.route('/')
def hello_world():
  return render_template('hello.html')

@app.route('/blogs/<int:blog_id>')
def blogs(blog_id):
  return render_template('blogs.html', blog_id=blog_id)  
  
app.run(host='0.0.0.0', port=8080)


In [16]:
from flask import Flask, render_template, request, jsonify, redirect, url_for
import json
from random import *
from math import *

import ntplib
from time import ctime

def get_ntp_time():
    c = ntplib.NTPClient()
    try:
        response = c.request('pool.ntp.org', version=3)
        return response.tx_time
    except ntplib.NTPException as e:
        print(f"NTPException: {e}")
        return None

app = Flask('app')

@app.route('/getTimerSetData')
def get_timerSetData():
    with open("params5.json") as f:
        data = json.load(f)
        return data

@app.route('/get_time')
def request_time():
    ntp_time = get_ntp_time()
    return jsonify({"time": ntp_time})

# @app.route('/')
# def homepage():
#     return render_template("form.html", len=4)

@app.route('/data/', methods=['POST','GET'])
def data():
    if request.method == 'GET':
        with open("params2.json") as f:
            data = json.load(f)
            print(data)
            return data
        return f"The URL /data is accessed directly Try going to '/form' to submit form."
    if request.method == 'POST':
        form_data = request.get_json()
        print("post")
        print(request.get_json())

        f = open("params2.json", "w")
        f.write(json.dumps(request.get_json(), indent=4))
        f.close()

        return render_template('data.html', form_data=form_data)
myform=[]
# @app.route('/jsondata/', methods=['GET','POST'])
@app.route('/', methods=['GET','POST'])
def jsondata():
    # ntptime = request_time()
    ntp_time = get_ntp_time()
    print("ntptime=", ntp_time)
    if request.method == 'GET':
        print("GET")
        # params3.json (old format that worked with populateDataFull). params5.json works with populateDataFull2
        with open("params5.json") as f:
            jsondata = json.load(f)
            # print(jsondata)
            return render_template('jsondata.html', jsondata = jsondata, ntp_time = ntp_time)

    if request.method == 'POST':
        print('POST')

        try:
            data = request.get_json()
            print(data)

            with open("params5.json", "w") as f:
                f.write(json.dumps(request.get_json(), indent=4))

            with open("params5.json") as f:
                jsondata = json.load(f)
                return jsonify(jsondata=jsondata, ntptime=ntp_time)
        except Exception as e:
            return jsonify({"message": str(e)})
        print('POST')
        myform=request.form
        print(request.form)
        return '<h1>successfully posted</h1>'

@app.route('/getdata')
def getdata():
    ntp_time = get_ntp_time()
    with open("params5.json") as f:
        jsondata = json.load(f)
        return jsonify(jsondata=jsondata, ntptime = ntp_time)
    
@app.route('/postdata', methods=['GET','POST'])
def postdata():
    if request.method == 'POST':
        form_data = request.form
        print(form_data)
        # print("post")
        # print(request.get_json())
        with open("params4.json", "w") as f:
            f.write(json.dumps(form_data))
        return "<p>results saved in params4.json</p>"
        
@app.route('/generatedata/', methods=['GET'])
def generatedata():
    data={}
    carr = []
    for c in range(8):
        darr=[]
        for d in range(7):
            parr=[]
            for p in range(4):
                parr.append(generateTime())
            darr.append(parr)
        carr.append(darr)
    data["data"] = carr
    with open('params3.json',"w") as f:
        f.write(json.dumps(data))
    return "<p>results generated in params3.json</p>"

@app.route('/schedule', methods=['GET'])
@app.route('/timerset', methods=['GET'])
def home():
    return redirect(url_for('jsondata'))
    return redirect("http://192.168.1.115:8080/")

def generateTime():
    t0 = twoDigit(24) + ":" + twoDigit(60)
    t1 = twoDigit(24) + ":" + twoDigit(60)
    tarr=[]
    tarr.append(t0)
    tarr.append(t1)
    return tarr

def twoDigit(r):
    t=floor(random()*r)
    t=str(int(t))
    t=t.zfill(2)
    return t

app.run(host='0.0.0.0', port=8080)

 * Serving Flask app 'app'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.1.115:8080
Press CTRL+C to quit
192.168.1.115 - - [16/Nov/2023 15:49:21] "GET /schedule HTTP/1.1" 302 -
192.168.1.115 - - [16/Nov/2023 15:49:21] "GET / HTTP/1.1" 200 -
192.168.1.115 - - [16/Nov/2023 15:49:21] "GET /static/css/style.css HTTP/1.1" 200 -
192.168.1.115 - - [16/Nov/2023 15:49:21] "GET /static/js/script.js HTTP/1.1" 200 -


ntptime= 1700178561.5488434
GET


192.168.1.115 - - [16/Nov/2023 15:49:33] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [16/Nov/2023 15:49:43] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [16/Nov/2023 15:49:53] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [16/Nov/2023 15:50:03] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [16/Nov/2023 15:50:13] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [16/Nov/2023 15:50:23] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [16/Nov/2023 15:50:33] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [16/Nov/2023 15:50:43] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [16/Nov/2023 15:50:53] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [16/Nov/2023 15:51:03] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [16/Nov/2023 15:51:13] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [16/Nov/2023 15:51:24] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [16/Nov/2023 15:51:34] "GET /get_time HTTP/1.1" 200 -
192.168.1.115 - - [16/Nov/2023 15:51:44] "GET /get_time HTTP/1.1

In [ ]:
def myMap(i):
    ii=4*(i-1)
    return 1+ii%20+floor(ii/20)

for x in range(1,21):
    print(f'{x}, {myMap(x)}')

In [ ]:
y=10
f'he{y}'
'{}'.format(y)
for y in range(3):
    f'hello {y}'

In [ ]:
from flask import Flask, render_template, jsonify

app = Flask('app')
mydata=''
@app.route('/atticus/')
def atticus():
    return 'hello atticus'

# SPA example 1: https://github.com/praveen-me/simple-vanila-router
@app.route('/')
def problem1():
    return render_template('problem1.html')

# SPA example 2: https://github.com/zathio/vanilla-spa
@app.route('/spa')
def problem2():
    return render_template('problem2.html')

@app.route('/data/', methods=['GET','POST'])
def problem3():
    if request.method == 'GET':
        return render_template('problem3.html')
    try:
        data = request.get_json()
        print(data)
        data2 = {"message":"bye"}
        return jsonify({"data": data2})
    except Exception as e:
        return jsonify({"message": str(e)})
    print(request.form)
    mydata=request.form
    return render_template('problem3.html')

app.run(host='0.0.0.0', port=8081)

In [ ]:
print(mydata.data)

In [ ]:
print(myform)

In [ ]:
for i in range(100):
    print('{} {}' , (i,i*i*i))

In [ ]:
import ntplib
from time import ctime
c = ntplib.NTPClient()
response = c.request('pool.ntp.org', version=3)
ctime(response.tx_time)

In [ ]:
import ntplib
from time import ctime
c = ntplib.NTPClient()
response = c.request('pool.ntp.org', version=3)
ctime(response.tx_time)

In [ ]:
from math import pi, pow
def findM(T,R1,R2):
    d=R1+R2
    G=6.673e-11 
    s=1.991e30  # sun mass, kg
    k=pow(2*pi/T*d,2)/G
    m1=k*R2
    m2=k*R1
    return (m1,m2,m1/s,m2/s)

T=40*365*24*3600 # 40 years
R1=6e11
R2=9e11
(m1,m2,m1_s,m2_s) = findM(T,R1,R2)
print(f'm1={m1:.3e}kg or {m1_s:.3f} solar mass, m2={m2:.3e}kg or {m2_s:.3f} solar mass')

In [ ]:
jsondata= {'timerSetData': {'c1d5p0o0': '13:00', 'c1d5p0o1': '14:00'}}
ntptime={'ntptime':'12345'}
jsondata.update(ntptime)
print(jsondata)